# COMS W1002 Computing in Context: Computational Finance  
## Professor Karl Sigman
## Option Pricing Using Monte Carlo Simulation

__Problem 1:__ For the values of $p=0.5$ and $p=0.75$ obtain an estimate of $p$ by using Monte Carlo Simulation with $n=100, n=1000, n=10,000$ You will generate $n$ _iid_ Bernoulli $(p)$ random variables $B_1, ..., B_n$ and use as the estimate $$ \hat{p}= \frac{1}{n}\sum_{i=1}^{n}B_i $$ 
This is to show you (convince you) how accurate the Monte Carlo method can be, and how the Strong Law of Large Numbers (SLLN) works in practice.

In [36]:
import random

def Bernoulli(p,n):
    counter = 0
    for i in range (n):
        if random.random()<p:
            counter+=1
    return counter/n

print("Feel free to run the function several times:")
print(Bernoulli(0.5,100))
print(Bernoulli(0.5,1000))
print(Bernoulli(0.5,10000))
print(Bernoulli(0.75,100))
print(Bernoulli(0.75,1000))
print(Bernoulli(0.75,10000))

Feel free to run the function several times:
0.55
0.507
0.4935
0.81
0.754
0.7425


__Problem 2:__ Recall the formula for computing the price $C_0$ of an option (derivative of the BLM stock prices) That yields a payoff at time $T$, denote by $C_T$:
$$ C_0 = \frac{1}{(1+r)^T}E^*(C_T), $$  

where $*$ refers to the fact that we must use the value $p^*$ instead of the original (real) $P$ for the up/down probability of the BLM. (The real value of $P$ is not needed for priciing.) Also recall that for $C_T = (S_T - K)^+$, the European call option, the expected value, $E^*(S_T-K)^+$ can be computed explicitly yielding the famous Black-Scholes-Merton option pricing formula:  
$$ C_0 = \frac{1}{(1+r)^T}\sum_{k=0}^{T} \left( \begin{array}{c}
T \\ k  \end{array} \right )(p^*)^k(1-p^*)^{T-k}(u^k d^{T-k}S_0-K)^+ . $$  

You are to use this formula to exactly obtain the price on the one hand, and then use Monte Carlo simulation on the other hand to compare and thus see how accurate the Monte Carlo method can be. 

Here are the parameters to use: $T = 10, r = 0.05, u = 1.15, d= 1.01, p*, S_0 = 50, K = 70$. Recall that 
$$ p^* = \frac{1+r-d}{u-d} . $$  

For the Monte Carlo, use $n=100, n=1000, n=10,000$ _iid_ copies of $C_T$ (for averaging) to see how it gets more accurate as $n$ increases.

In [77]:
#don't have to do factorials with .comb
import math

def combination(T,k):
    return math.factorial(T)/(math.factorial(T-k)*math.factorial(k))

def Cnott(T,r,u,d,S,K):
    p_star = p(r,u,d)
    C0 = 0
    for k in range(T):
        C0 += combination(T,k) * (p_star**k) * (1-p_star)**(T-k)* max((u**k)*(d**(T-k)) * (S) - (K), 0) #Max for plus exponent
    return C0 / (1 + r)**T 
  

#"def Y" given to us in document:
  
def Y(p,u,d):
#1 evaluates to true and 0 to false
    if Bernoulli(p,1):
        return u
    else:
        return d

# "def BLM" iven to us in document:
def BLM(S0,n,p,u,d):
#Initialize an empty array
    BLMpath = [0 for k in range(n+1)]
#Set the first entry to our initial stock value S_0=S0
    BLMpath[0] = S0
#Generate all $n$ of our iid Y samples Y_1,Y_2,...Y_n at once
    Ysamples = [Y(p,u,d) for k in range(n)]
    S = S0
    for k in range(1,n+1):
        S *= Ysamples[k-1]
        BLMpath[k] = S
    return BLMpath

def p(r,u,d):
    p_star = (1+r-d)/(u-d)
    return p_star


def monteC0(T,r,u,d,S,K,n):
    Cnottnew = 0
    p_star = p(r,u,d)
    for k in range(n):
        S_T = BLM(S, T, p_star, u, d)[-1]
        Cnottnew += max(S_T - K, 0)
    return (1 / ((1+r)**T)) * (1/n) * Cnottnew


print(Cnott(10,0.05,1.15,1.01,50,70))
    
#print(monteC0(10,0.05,1.15,1.01,50,70,10))
#print(monteC0(10,0.05,1.15,1.01,50,70,100))
#print(monteC0(10,0.05,1.15,1.01,50,70,1000))
print(monteC0(10,0.05,1.15,1.01,50,70,10000))



7.943105102106478
7.879471728154839


__Problem 3__: With the same parameters as in Problem 2, price the _Asian call option:_ The payoff $C_T$ at atime $T$ is based on the average value of the stock over the $T$ time units:
$$ C_T = \left( \frac{1}{T} (\sum_{i=1}^{T} S_i)-K \right)^+ $$  

Use $n=100, n=1000, n=10,000$ _iid_ copies of $C_T$ (for averaging).

In [109]:

def asianmonteC0(T,r,u,d,S,K,n):
    Cnottnew = 0
    p_star = p(r,u,d)
    for k in range(n):
        val = max(sum(BLM(S, T, p_star, u, d))/T - K,0)
        Cnottnew += val
    return (1 / ((1+r)**T)) * (1/n) * Cnottnew


print(asianmonteC0(10,0.05,1.15,1.01,50,70,100))
print(asianmonteC0(10,0.05,1.15,1.01,50,70,1000))
print(asianmonteC0(10,0.05,1.15,1.01,50,70,10000))

2.1743123003596625
2.4248674282232487
2.3139334939634484


__Problem 4__: With the same parameters as in Problem 2, but the additional parameters $n_1 = 4, n_2 =6,$ and $b=66$: price a _down and out barrier option_, that has payoff at time $T$ of  

$$ C_T = (S_T -K)^+ I \{ S_{n_1} \geq b, S_{n_2} \geq b \}.$$ 

Use $n=100, n=1000, n=10,000$ _iid_ copies of $C_T$ (for averaging).

In the above, recall that for any event $A$, $I\{A\}$ denotes the _indicator_ random variable defined by  
$$ I\{A\} = \left\{ \begin{array}{ll} 1 & \mbox{if $A$ occurs,} \\ 0 & \mbox{if $A$ does not occur.} \end{array} \right. $$

  
Here, $A = \{S_{n_1} \geq b, S_{n_2} \geq b\}$.


In [107]:
def downout(T,r,u,d,S,K,n,n1,n2,b):
    Cnottnew = 0
    p_star = p(r,u,d)
    for k in range(n):
        S_s = BLM(S, T, p_star, u, d)
        S_T = S_s[-1]
        S_n1 = S_s[n1]
        S_n2 = S_s[n2-]
        I = 1 if S_n1 >= b and S_n2 >= b else 0
        Cnottnew += max(S_T - K, 0) * I
    return (1 / ((1+r)**T)) * (1/n) * Cnottnew


print(downout(10,0.05,1.15,1.01,50,70,100,4,6,66))
print(downout(10,0.05,1.15,1.01,50,70,1000,4,6,66))
print(downout(10,0.05,1.15,1.01,50,70,10000,4,6,66))

4.505344377363382
4.643074936378376
4.672880477223146


In [102]:
print("when he knows how to code <3")

when he knows how to code <3
